<a href="https://colab.research.google.com/github/bharathreddy-2802/BharathSimhaReddy_INFO5731_Fall2024/blob/main/Samala_BharathSimhaReddy_Exercise_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 In-class Exercise 3**

The purpose of this exercise is to explore various aspects of text analysis, including feature extraction, feature selection, and text similarity ranking.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

**Total points**: 40

**Deadline**: This in-class exercise is due at the end of Friday, at 11:59 PM.

**Late submissions will have a penalty of 10% of the marks for each day of late submission. , and no requests will be answered. Manage your time accordingly.**

**Please check that the link you submitted can be opened and points to the correct assignment.**

## Question 1 (10 Points)
Describe an interesting **text classification or text mining task** and explain what kind of features might be useful for you to build the machine learning model. List your features and explain why these features might be helpful. You need to list at least five different types of features. **Your dataset must be text.**

In [1]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):

'''

The interest will be in spam detection. This is common in emails where you get spam messages which are deemed dangerous
as they may carry viruses or be used to steal personal data.

 classifying emails as spam or not spam will be the goal in this question. The features include:
1. Bag of Words: Certain keywords frequently appear in spam emails.
2. Email Length: Spam emails might be shorter or repetitive.
3. Punctuation Count: Spam often uses excessive punctuation.
4. Link Presence: Spam emails often contain links encouraging clicks.
5. Subject Line Length: Unusually long subject lines might indicate spam.





'''

'\n\nThe interest will be in spam detection. This is common in emails where you get spam messages which are deemed dangerous\nas they may carry viruses or be used to steal personal data.\n\n classifying emails as spam or not spam will be the goal in this question. The features include:\n1. Bag of Words: Certain keywords frequently appear in spam emails.\n2. Email Length: Spam emails might be shorter or repetitive.\n3. Punctuation Count: Spam often uses excessive punctuation.\n4. Link Presence: Spam emails often contain links encouraging clicks.\n5. Subject Line Length: Unusually long subject lines might indicate spam.\n\n\n\n\n\n'

## Question 2 (10 Points)
Write python code to extract these features you discussed above. You can collect a few sample text data for the feature extraction.

In [2]:
# You code here (Please add comments in the code):
import pandas as pd # imports the pandas library and aliases it as 'pd'

# email data from my inbox
email_data = [

              "Bet and win big!!!!",
              "Win a free iPhone now!!!",
              "Here is the agenda for tomorrow's meeting",
              "Click here to claim your $1000 prize."


              ]

#This function will help to extract the features of the above set of emails
def extract_email_features(emails):
    features = []

    for email in emails:
        email_features = {}
        email_features['word_count'] = len(email.split())
        email_features['punctuation_count'] = sum([1 for char in email if char in '?!.,'])
        email_features['link_presence'] = int("click" in email.lower())
        email_features['subject_length'] = len(email)
        features.append(email_features)

    return pd.DataFrame(features)

email_features_df = extract_email_features(email_data)
print(email_features_df)



   word_count  punctuation_count  link_presence  subject_length
0           4                  4              0              19
1           5                  3              0              24
2           7                  0              0              41
3           7                  1              1              37


## Question 3 (10 points):
Use any of the feature selection methods mentioned in this paper "Deng, X., Li, Y., Weng, J., & Zhang, J. (2019). Feature selection for text classification: A review. Multimedia Tools & Applications, 78(3)."

Select the most important features you extracted above, rank the features based on their importance in the descending order.

In [3]:
# You code here (Please add comments in the code):

#Importing Packages
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_extraction.text import TfidfVectorizer

# email data from my inbox
email_data = [

              "Bet and win big!!!!",
              "Win a free iPhone now!!!",
              "Here is the agenda for tomorrow's meeting",
              "Click here to claim your $1000 prize."


              ]

# Labels for the emails (1 = is spam, 0 = not spam)
y = [1,1, 0, 1]

# Initializing TF-IDF vectorizer
vectorizer = TfidfVectorizer()
#Initializing fit  on the email data
X = vectorizer.fit_transform(email_data)

# Performing feature selection using SelectKBest and Chi-Square test
selector = SelectKBest(score_func=chi2, k=2)  # Selecting top 2 features;k=2
X_new = selector.fit_transform(X, y)

# Getting feature scores and ranking the features by their Chi-Square score
feature_scores = selector.scores_
feature_names = vectorizer.get_feature_names_out()
sorted_features = sorted(zip(feature_scores, feature_names), reverse=True)

# Displaying the ranked features
print("Ranked Features (Chi-Square Scores):")
for score, feature in sorted_features:
    print(f"Feature: {feature} Score: {score}")


Ranked Features (Chi-Square Scores):
Feature: tomorrow Score: 1.1658428778939514
Feature: the Score: 1.1658428778939514
Feature: meeting Score: 1.1658428778939514
Feature: is Score: 1.1658428778939514
Feature: for Score: 1.1658428778939514
Feature: agenda Score: 1.1658428778939514
Feature: win Score: 0.27619250077725976
Feature: here Score: 0.2042586479887876
Feature: now Score: 0.17515758308802193
Feature: iphone Score: 0.17515758308802193
Feature: free Score: 0.17515758308802193
Feature: big Score: 0.17515758308802193
Feature: bet Score: 0.17515758308802193
Feature: and Score: 0.17515758308802193
Feature: your Score: 0.12953809754377232
Feature: to Score: 0.12953809754377232
Feature: prize Score: 0.12953809754377232
Feature: click Score: 0.12953809754377232
Feature: claim Score: 0.12953809754377232
Feature: 1000 Score: 0.12953809754377232


## Question 4 (10 points):
Write python code to rank the text based on text similarity. Based on the text data you used for question 2, design a query to match the most relevant docments. Please use the BERT model to represent both your query and the text data, then calculate the cosine similarity between the query and each text in your data. Rank the similary with descending order.

In [4]:
# You code here (Please add comments in the code):

# Installing the transformers library as it is needed in this question.
!pip install transformers

from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Loading BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# email data from my inbox
email_data = [

              "Bet and win big!!!!",
              "Win a free iPhone now!!!",
              "Here is the agenda for tomorrow's meeting",
              "Click here to claim your $1000 prize."


              ]

# Query definition for similarity matching
query = "Congratulations! You won a free prize. Click here to claim."

# A helper function to get BERT embeddings for a given text
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# BERT embedding for the query I defined above
query_embedding = get_bert_embedding(query)

# Get cosine similarities between the query and each email
similarities = []
for email in email_data:
    email_embedding = get_bert_embedding(email)
    similarity = cosine_similarity(query_embedding, email_embedding)[0][0]
    similarities.append((email, similarity))

# Arranging the emails by similarity in descending order
similarities.sort(key=lambda x: x[1], reverse=True)

# Showing the ranked emails based on similarity to the query
print("Ranked Emails by Similarity: ")
for email, score in similarities:
    print(f"Email: {email} | Similarity Score: {score}")




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Ranked Emails by Similarity: 
Email: Click here to claim your $1000 prize. | Similarity Score: 0.8443804383277893
Email: Win a free iPhone now!!! | Similarity Score: 0.7109931707382202
Email: Bet and win big!!!! | Similarity Score: 0.6581276655197144
Email: Here is the agenda for tomorrow's meeting | Similarity Score: 0.6426362991333008


# Mandatory Question

**Important: Reflective Feedback on this exercise**

Please provide your thoughts and feedback on the exercises you completed in this assignment. Consider the following points in your response:

Learning Experience: Describe your overall learning experience in working on extracting features from text data. What were the key concepts or techniques you found most beneficial in understanding the process?

Challenges Encountered: Were there specific difficulties in completing this exercise?

Relevance to Your Field of Study: How does this exercise relate to the field of NLP?

**(Your submission will not be graded if this question is left unanswered)**



In [ ]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):

'''

The feature extraction from text data was an interesting task. I found the vectorizer, the tokenizer and the Bert Model important for a serious
text data analysis. I found out that for text data extraction, there is the need to prepare data first. The notable challenges with the exercise was the warnings
that were coming from the helper libraries that I was using. Though I believe that the output was not affected. The exercise is squarely related to NLP and
it provided a great platform to showcase the skills acquired.



'''

'\n\nThe feature extraction from text data was an interesting task. I found the vectorizer, the tokenizer and the Bert Model important for a serious\ntext data analysis. I found out that for text data extraction, there is the need to prepare data first. The notable challenges with the exercise was the warnings\nthat were coming from the helper libraries that I was using. Though I believe that the output was not affected. The exercise is squarely related to NLP and \nit provided a great platform to showcase the skills acquired.\n\n\n\n'